In [1]:
import re
import numpy
import matplotlib.pyplot as plot
import pickle

from pymongo import MongoClient

from textblob import TextBlob

import nltk
nltk.download('wordnet')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet,  words, stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora

from tqdm import tqdm
from pympler import asizeof

from IPython.display import display

[nltk_data] Downloading package wordnet to /Users/ram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /Users/ram/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ram/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
english_words = set(words.words())
english_stopwords = set(stopwords.words('english'))

lemmatize = WordNetLemmatizer().lemmatize


In [3]:
# pos_tags: r - adverb, j - adjective, n - noun, v - verb, 
def tokenize(text):
    text = text.lower()
    
    re_word = re.compile('[a-z]+')
    
    tokens = []
    for token, pos in pos_tag(word_tokenize(text)):
        
        if not bool(re_word.match(token)):
            continue

        lemma = None
        if pos[0] in ['A', 'N', 'R', 'V']:
            lemma = lemmatize(token, pos[0].lower())
        else:
            if token not in english_words and wordnet.morphy(token) is None:
                continue
            lemma = token
            
        tokens.append(lemma)
    return tokens

In [4]:
def clean(tokens):
    tokens = [token for token in tokens if token not in english_stopwords]
    tokens = [token for token in tokens if len(token) > 2]
    return tokens

In [5]:
mongoClient = MongoClient(port=27017)
db = mongoClient.yelp

In [6]:
tokenized_docs = []
progress = tqdm(total=767985, leave=True, position=0)
cursor = db.reviews_sub.find({}, {'tokens': 1}).sort('_id', 1)
for doc in cursor:
    progress.update(1)
    if 'tokens' in doc:
        tokenized_docs.append(doc['tokens'])

progress.refresh()

100%|██████████| 767985/767985 [00:22<00:00, 38489.45it/s]

True

In [8]:
%%time
dictionary = corpora.Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(tokenized_doc) for tokenized_doc in tokenized_docs]

dictionary.save('dictionary_sub.pkl')
with open('corpus_sub.pkl', 'wb') as file:
    pickle.dump(corpus, file)

CPU times: user 1min 36s, sys: 4.42 s, total: 1min 40s
Wall time: 1min 41s


In [ ]:
dictionary = None
corpus = None

with open('dictionary_sub.pkl', 'rb') as file:
    dictionary = pickle.load(file)
    
with open('corpus_sub.pkl', 'rb') as file:
    corpus = pickle.load(file)

In [17]:
import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [10]:
%%time
num_topics = 30

model = LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=40)
model.save('model_.h5')

2020-04-10 14:47:08,845 : INFO : using symmetric alpha at 0.03333333333333333
2020-04-10 14:47:08,846 : INFO : using symmetric eta at 0.03333333333333333
2020-04-10 14:47:08,882 : INFO : using serial LDA version on this node
2020-04-10 14:47:09,415 : INFO : running online LDA training, 30 topics, 40 passes over the supplied corpus of 767985 documents, updating every 30000 documents, evaluating every ~300000 documents, iterating 50x with a convergence threshold of 0.001000
2020-04-10 14:47:09,417 : INFO : training LDA model using 15 processes
2020-04-10 14:47:09,542 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/767985, outstanding queue size 1
2020-04-10 14:47:10,765 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/767985, outstanding queue size 2
2020-04-10 14:47:10,775 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/767985, outstanding queue size 3
2020-04-10 14:47:10,785 : INFO : PROGRESS: pass 0, dispatched chunk 

2020-04-10 14:47:23,989 : INFO : PROGRESS: pass 0, dispatched chunk #50 = documents up to #102000/767985, outstanding queue size 32
2020-04-10 14:47:24,103 : INFO : PROGRESS: pass 0, dispatched chunk #51 = documents up to #104000/767985, outstanding queue size 33
2020-04-10 14:47:25,261 : INFO : PROGRESS: pass 0, dispatched chunk #52 = documents up to #106000/767985, outstanding queue size 33
2020-04-10 14:47:25,615 : INFO : PROGRESS: pass 0, dispatched chunk #53 = documents up to #108000/767985, outstanding queue size 33
2020-04-10 14:47:26,433 : INFO : PROGRESS: pass 0, dispatched chunk #54 = documents up to #110000/767985, outstanding queue size 33
2020-04-10 14:47:27,212 : INFO : PROGRESS: pass 0, dispatched chunk #55 = documents up to #112000/767985, outstanding queue size 33
2020-04-10 14:47:28,116 : INFO : PROGRESS: pass 0, dispatched chunk #56 = documents up to #114000/767985, outstanding queue size 33
2020-04-10 14:47:29,132 : INFO : PROGRESS: pass 0, dispatched chunk #57 = do

2020-04-10 14:47:54,609 : INFO : topic #23 (0.033): 0.022*"food" + 0.014*"buffet" + 0.011*"come" + 0.010*"good" + 0.010*"n't" + 0.009*"get" + 0.009*"would" + 0.008*"great" + 0.008*"try" + 0.008*"like"
2020-04-10 14:47:54,612 : INFO : topic #29 (0.033): 0.018*"food" + 0.018*"get" + 0.017*"buffet" + 0.014*"n't" + 0.013*"room" + 0.012*"wait" + 0.008*"good" + 0.008*"like" + 0.007*"hotel" + 0.007*"line"
2020-04-10 14:47:54,615 : INFO : topic #9 (0.033): 0.022*"burger" + 0.017*"buffet" + 0.014*"good" + 0.011*"fry" + 0.011*"n't" + 0.010*"vega" + 0.009*"one" + 0.009*"food" + 0.009*"place" + 0.009*"best"
2020-04-10 14:47:54,617 : INFO : topic #5 (0.033): 0.014*"good" + 0.011*"food" + 0.010*"vega" + 0.010*"buffet" + 0.009*"get" + 0.008*"place" + 0.008*"like" + 0.007*"also" + 0.007*"n't" + 0.006*"best"
2020-04-10 14:47:54,630 : INFO : topic diff=0.081908, rho=0.147442
2020-04-10 14:47:54,643 : INFO : PROGRESS: pass 0, dispatched chunk #92 = documents up to #186000/767985, outstanding queue size 3

2020-04-10 14:48:23,394 : INFO : PROGRESS: pass 0, dispatched chunk #130 = documents up to #262000/767985, outstanding queue size 33
2020-04-10 14:48:24,169 : INFO : PROGRESS: pass 0, dispatched chunk #131 = documents up to #264000/767985, outstanding queue size 33
2020-04-10 14:48:24,850 : INFO : PROGRESS: pass 0, dispatched chunk #132 = documents up to #266000/767985, outstanding queue size 33
2020-04-10 14:48:25,583 : INFO : PROGRESS: pass 0, dispatched chunk #133 = documents up to #268000/767985, outstanding queue size 33
2020-04-10 14:48:26,464 : INFO : PROGRESS: pass 0, dispatched chunk #134 = documents up to #270000/767985, outstanding queue size 33
2020-04-10 14:48:26,974 : INFO : PROGRESS: pass 0, dispatched chunk #135 = documents up to #272000/767985, outstanding queue size 33
2020-04-10 14:48:27,822 : INFO : PROGRESS: pass 0, dispatched chunk #136 = documents up to #274000/767985, outstanding queue size 33
2020-04-10 14:48:28,577 : INFO : merging changes from 30000 documents

2020-04-10 14:48:51,828 : INFO : topic #25 (0.033): 0.025*"n't" + 0.011*"good" + 0.010*"get" + 0.010*"like" + 0.009*"try" + 0.008*"would" + 0.008*"food" + 0.008*"dish" + 0.008*"come" + 0.007*"really"
2020-04-10 14:48:51,839 : INFO : topic diff=0.045409, rho=0.090909
2020-04-10 14:48:51,853 : INFO : PROGRESS: pass 0, dispatched chunk #167 = documents up to #336000/767985, outstanding queue size 33
2020-04-10 14:48:52,510 : INFO : PROGRESS: pass 0, dispatched chunk #168 = documents up to #338000/767985, outstanding queue size 32
2020-04-10 14:48:53,152 : INFO : PROGRESS: pass 0, dispatched chunk #169 = documents up to #340000/767985, outstanding queue size 32
2020-04-10 14:48:53,678 : INFO : PROGRESS: pass 0, dispatched chunk #170 = documents up to #342000/767985, outstanding queue size 33
2020-04-10 14:48:54,426 : INFO : PROGRESS: pass 0, dispatched chunk #171 = documents up to #344000/767985, outstanding queue size 33
2020-04-10 14:48:55,551 : INFO : PROGRESS: pass 0, dispatched chunk 

...

2020-04-10 18:02:09,143 : INFO : merging changes from 30000 documents into a model of 767985 documents
2020-04-10 18:02:09,415 : INFO : topic #10 (0.033): 0.105*"drink" + 0.104*"bar" + 0.080*"beer" + 0.033*"hour" + 0.031*"happy" + 0.029*"bartender" + 0.023*"cocktail" + 0.019*"night" + 0.019*"great" + 0.012*"selection"
2020-04-10 18:02:09,417 : INFO : topic #4 (0.033): 0.063*"good" + 0.045*"food" + 0.038*"price" + 0.032*"place" + 0.027*"great" + 0.020*"pretty" + 0.019*"service" + 0.018*"really" + 0.015*"nice" + 0.014*"little"
2020-04-10 18:02:09,421 : INFO : topic #19 (0.033): 0.058*"best" + 0.039*"ever" + 0.033*"restaurant" + 0.023*"experience" + 0.022*"chef" + 0.015*"one" + 0.014*"kitchen" + 0.014*"food" + 0.014*"amazing" + 0.014*"every"
2020-04-10 18:02:09,423 : INFO : topic #24 (0.033): 0.014*"make" + 0.012*"work" + 0.012*"great" + 0.011*"look" + 0.011*"job" + 0.010*"time" + 0.010*"get" + 0.010*"massage" + 0.010*"experience" + 0.010*"take"
2020-04-10 18:02:09,426 : INFO : topic #5 (

2020-04-10 18:02:32,782 : INFO : PROGRESS: pass 39, dispatched chunk #291 = documents up to #584000/767985, outstanding queue size 32
2020-04-10 18:02:33,404 : INFO : PROGRESS: pass 39, dispatched chunk #292 = documents up to #586000/767985, outstanding queue size 32
2020-04-10 18:02:34,210 : INFO : PROGRESS: pass 39, dispatched chunk #293 = documents up to #588000/767985, outstanding queue size 32
2020-04-10 18:02:34,812 : INFO : PROGRESS: pass 39, dispatched chunk #294 = documents up to #590000/767985, outstanding queue size 32
2020-04-10 18:02:35,548 : INFO : PROGRESS: pass 39, dispatched chunk #295 = documents up to #592000/767985, outstanding queue size 32
2020-04-10 18:02:36,108 : INFO : PROGRESS: pass 39, dispatched chunk #296 = documents up to #594000/767985, outstanding queue size 32
2020-04-10 18:02:36,782 : INFO : PROGRESS: pass 39, dispatched chunk #297 = documents up to #596000/767985, outstanding queue size 32
2020-04-10 18:02:37,386 : INFO : PROGRESS: pass 39, dispatched

2020-04-10 18:02:59,871 : INFO : topic #19 (0.033): 0.059*"best" + 0.040*"ever" + 0.033*"restaurant" + 0.023*"experience" + 0.021*"chef" + 0.015*"one" + 0.014*"food" + 0.014*"kitchen" + 0.014*"every" + 0.014*"amazing"
2020-04-10 18:02:59,874 : INFO : topic #24 (0.033): 0.015*"make" + 0.012*"work" + 0.012*"great" + 0.011*"look" + 0.011*"job" + 0.010*"time" + 0.010*"get" + 0.010*"experience" + 0.010*"take" + 0.009*"massage"
2020-04-10 18:02:59,876 : INFO : topic #14 (0.033): 0.064*"n't" + 0.030*"like" + 0.025*"place" + 0.021*"get" + 0.020*"would" + 0.019*"really" + 0.018*"good" + 0.016*"star" + 0.016*"think" + 0.013*"say"
2020-04-10 18:02:59,879 : INFO : topic #28 (0.033): 0.056*"always" + 0.050*"time" + 0.033*"love" + 0.027*"place" + 0.026*"year" + 0.025*"get" + 0.021*"n't" + 0.019*"every" + 0.019*"one" + 0.019*"never"
2020-04-10 18:02:59,891 : INFO : topic diff=0.022847, rho=0.048565
2020-04-10 18:02:59,897 : INFO : PROGRESS: pass 39, dispatched chunk #332 = documents up to #666000/767

2020-04-10 18:03:23,332 : INFO : PROGRESS: pass 39, dispatched chunk #369 = documents up to #740000/767985, outstanding queue size 32
2020-04-10 18:03:23,952 : INFO : PROGRESS: pass 39, dispatched chunk #370 = documents up to #742000/767985, outstanding queue size 32
2020-04-10 18:03:24,555 : INFO : PROGRESS: pass 39, dispatched chunk #371 = documents up to #744000/767985, outstanding queue size 32
2020-04-10 18:03:25,240 : INFO : PROGRESS: pass 39, dispatched chunk #372 = documents up to #746000/767985, outstanding queue size 32
2020-04-10 18:03:25,890 : INFO : PROGRESS: pass 39, dispatched chunk #373 = documents up to #748000/767985, outstanding queue size 32
2020-04-10 18:03:26,623 : INFO : PROGRESS: pass 39, dispatched chunk #374 = documents up to #750000/767985, outstanding queue size 32
2020-04-10 18:03:27,174 : INFO : PROGRESS: pass 39, dispatched chunk #375 = documents up to #752000/767985, outstanding queue size 32
2020-04-10 18:03:27,810 : INFO : PROGRESS: pass 39, dispatched

CPU times: user 1h 39min 27s, sys: 1h 1min 30s, total: 2h 40min 57s
Wall time: 3h 16min 49s


In [20]:
display(model.print_topics())

[(6,
  '0.044*"place" + 0.040*"vega" + 0.037*"time" + 0.033*"recommend" + 0.032*"back" + 0.031*"definitely" + 0.030*"come" + 0.025*"try" + 0.024*"best" + 0.021*"would"'),
 (28,
  '0.054*"always" + 0.049*"time" + 0.033*"love" + 0.026*"place" + 0.026*"year" + 0.025*"get" + 0.021*"n\'t" + 0.020*"nail" + 0.019*"never" + 0.018*"one"'),
 (7,
  '0.044*"cream" + 0.043*"chocolate" + 0.037*"ice" + 0.031*"dessert" + 0.028*"cake" + 0.019*"sweet" + 0.016*"crepe" + 0.015*"try" + 0.013*"get" + 0.010*"strawberry"'),
 (26,
  '0.097*"great" + 0.059*"service" + 0.056*"food" + 0.041*"friendly" + 0.040*"place" + 0.039*"staff" + 0.035*"love" + 0.019*"awesome" + 0.016*"back" + 0.016*"atmosphere"'),
 (23,
  '0.078*"taco" + 0.029*"salsa" + 0.028*"mexican" + 0.023*"margarita" + 0.023*"chip" + 0.019*"order" + 0.018*"food" + 0.017*"corn" + 0.016*"burrito" + 0.016*"good"'),
 (19,
  '0.059*"best" + 0.040*"ever" + 0.032*"restaurant" + 0.023*"experience" + 0.022*"chef" + 0.015*"kitchen" + 0.015*"one" + 0.014*"food" +